In [ ]:
import language_processing as lang_process
import parse_institutional_data as parse_data
import entity_processing as entity_proc
import process_search_terms as process
import json
import os

In [41]:
from importlib import reload
reload(lang_process);
reload(parse_data);
reload(process);
reload(entity_proc)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vonbecker/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/vonbecker/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/vonbecker/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vonbecker/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/vonbecker/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
# read in and pre-process the exhibit keywords and phenomena

path = '../get_institutional_data/data/raw/exhibits.json'
phenomena = parse_data.get_keywords(path, 'phenomena')
proc_phenomena = [process.preprocess(phenomenon, 'phenomenon') for phenomenon in phenomena]
keywords = parse_data.get_keywords(path, 'keywords')
proc_keywords = [process.preprocess(keyword, 'keyword') for keyword in keywords]

In [ ]:
# join preprocessed keywords and phenomena, keep unique elements

all_search_terms = proc_phenomena + proc_keywords
unique_search_terms = []
[unique_search_terms.append(x) for x in all_search_terms if x not in unique_search_terms];

In [42]:
# create list of all search term data, including related terms and synsets
# save to disk

all_data = [process.get_all_data(term) for term in unique_search_terms]

path = '../get_institutional_data/data/'
if not os.path.exists(path):
    os.makedirs(path)

filepath = os.path.join(path, 'all_search_terms.json')
if os.path.exists(filepath):
    os.remove(filepath)

with open(filepath, 'w') as outfile:
    outfile.write(json.dumps(all_data, indent=2))